What we need is a small parcel to work on, with a nice contourplot with critical points and minima found after initiating local method and then a 3d plot. 

In [ ]:
include(joinpath(dirname(Base.find_package("Globtim")), "..", ".globtim", "notebook_setup.jl"))

In [ ]:
# Constants and Parameters
const n, a, b = 2, 5, 12
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = tref # Objective function

One may assume that when we have access to exact evaluations, we would want to have a small $L^2$-norm tolerance `tol_l2 = 5e-4` and high probability of computing an accurate discrete $L^2$-norm `alpha= 1/10`.

We need to also return the number of samples used to generate the sample set. It is annoying that the error goes up while the degree has increased.

In [ ]:
# rand_center = [2*rand()-1, 2*rand()-1]*.75; # Random center
# rand_center = [0.4694712315165298, -0.45637754560934185]
rand_center = [0.0, 0.0];

In [ ]:
d = 10 # Initial Degree 
SMPL = 100 # Number of samples
TR = test_input(f, 
                dim = n,
                center=rand_center,
                GN=SMPL, 
                sample_range=scale_factor, 
                degree_max = d+4
                )
pol_cheb = Constructor(TR, d, basis=:chebyshev)
pol_lege = Constructor(TR, d, basis=:legendre);

@polyvar(x[1:n]); # Define polynomial ring 

Solve the system of partial derivatives using `Msolve`. For degree `26` on tref centered at `rand_center = [0.4694712315165298, -0.45637754560934185]` and `a/b = 7//12`, `msolve runs single threaded in 5m 2.2 s. With 10 threads, it runs in 4m 48.2 s. 
The slow part of the process is the real root isolation it would seem. 

In [ ]:
df_cheb = solve_and_parse(pol_cheb, x, f, TR)
sort!(df_cheb, :z, rev=true)

Legendre polynomial: 8m49 s, `scale_factor = 1/3` and degree `30`, and 300 sample points.  

In [ ]:
# df_lege = solve_and_parse(pol_lege, x, f, TR, basis=:legendre)
# sort!(df_lege, :z, rev=true)

In [ ]:
using Optim
df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=0.01)

In [ ]:
using GLMakie
GLMakie.activate!(inline=false)  # Ensure GLMakie is active for 3D plots

In [ ]:
# df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR)
# df_lege, df_min_lege = analyze_critical_points(f, df_lege, TR)

In [ ]:
plot_polyapprox_rotate(pol_cheb, TR, df_cheb, df_min_cheb)

In [ ]:
fig1 = cairo_plot_polyapprox_levelset(pol_cheb, TR, df_cheb, df_min_cheb)

In [ ]:
# plot_polyapprox_levelset(pol_cheb, TR, df_cheb, df_min_cheb)

In [ ]:
# plot_polyapprox_levelset(pol_lege, TR, df_lege, df_min_lege)

In [ ]:
# GLMakie.closeall()

The trefethen example: 
Is about the 100 digit challenge, which goal was to compute the global minimizer of the `tref` function over the domain $[-1,1]^2$ and to refine the accuracy of that minimizer up to a 100 digits.
This function has ... many local minimizers in the interior of that domain. 
With `Globtim`, we attempt to approach all critical points.
To keep the degree of the approximant reasonable, we subdivide the domain into smaller squares and apply `Globtim` to each of them.

How do we measure the discrete error: Riemman sum, discrete $L^2$-norm, or the maximum error?
--> we verify the error on a separately generated uniform grid of $N = $ points.

We refine the candidate local minimizers we computed with msolve through a local optimization method in the `Optim` package in Julia. 
